In [ ]:
# pre-install packages required for PYTHON3
import sys
from IPython.core.display import display, HTML
# python3 
pkgs = ['numpy','or2ywtool','git+https://github.com/joseboveda/refine-client-py','git+https://github.com/jezcope/pyrefine']
for package in pkgs:
    try:
        import package
    except ImportError as error:
        # Install a pip package in the current Jupyter kernel
        !{sys.executable} -m pip install {package}
    except Exception as exception:
        # Output unexpected Exceptions.
        print(exception)



In [1]:
# read in data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_dish = pd.read_csv('../data/Dish.csv', index_col='id',encoding='latin1')



In [2]:
# show rows
df_dish.tail(10)

,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
id,,,,,,,,
515668,"Sparkling Wine: Longworth, Sparkling Catawba",NaN,1,1,0,0,0.0,0.0
515669,M. Y. Maderia,NaN,1,1,0,0,0.0,0.0
515670,Partridge & Bread Sauce,NaN,1,1,0,0,0.0,0.0
515671,Roast: Mutton & Currant Jelly,NaN,1,1,0,0,0.0,0.0
515672,Boiled: Calves head Bacon Brain Sauce,NaN,1,1,0,0,0.0,0.0
515673,Boiled: Corned beef & cabbage,NaN,1,1,0,0,0.0,0.0
515674,Boiled: Knuckle of Veal & Bacon,NaN,1,1,0,0,0.0,0.0
515675,Roast: Turkey & Cranberry Sauce,NaN,1,1,0,0,0.0,0.0
515676,"Claret: Chateau Larose, Cruse et Fils Freres",NaN,1,1,0,0,0.0,0.0


In [3]:
# normalize the dish name
def normalize_string(x):
    # normalize_string will remove whitespace and convert to lower
    tokens = x.strip().lower().split()
    result = ' '.join(filter(None, tokens))
    return result

df_dish['name2']=df_dish['name'].map(normalize_string)

In [4]:
# setup python-to-openrefine
from open.refine import refine,facet
import os


def list_projects(orefine):
    """Query the Refine server and list projects by ID: name."""
    projects = orefine.list_projects().items()

    def date_to_epoch(json_dt):
        """Convert a JSON date time into seconds-since-epoch."""
        return time.mktime(time.strptime(json_dt, '%Y-%m-%dT%H:%M:%SZ'))
    
    #projects.sort(key=lambda v: date_to_epoch(v[1]['modified']), reverse=True)
    for project_id, project_info in projects:
        print('{0:>14}: {1}'.format(project_id, project_info['name']))


     

In [5]:
# setup server/client communication

server = refine.RefineServer()
orefine = refine.Refine(server)


# list projects 
list_projects(orefine)
# open project 
KEY = '1837847158105'
project = orefine.open_project(KEY)


 2495423155658: jamesmb3_airbnb
 2263047142245: jamesmb3_airbnb
 2448365894229: jamesmb3_airbnb
 2430007378648: NYPL_MENU
 1914218214657: NYPL_MENUITEM
 1837847158105: NYPL_DISH
 2505595537055: jamesmb3_airbnb
 2219637648827: NYPL_MENUPAGE
 2484482200133: airbnb_dirty csv


In [6]:
# show columns
project.columns


['id',
 'name',
 'description',
 'menus_appeared',
 'times_appeared',
 'first_appeared',
 'last_appeared',
 'lowest_price',
 'highest_price']

In [7]:
# build facet

clusters=project.compute_clusters(column="name")

# failed - something wrong with the client...

Exception: server error: Unexpected character (''' (code 39)): was expecting either valid name character (for unquoted name) or double-quote (for quoted) to start field name
 at [Source: (String)"{'type': 'binning', 'function': 'fingerprint', 'params': {}, 'column': 'name'}"; line: 1, column: 3]

In [9]:
# try to build a fingerprint using similar algorithm 
import re
def fingerprint(x):
    alphanumeric_tokens = filter(None, re.split('\W', x))
    seen = set()
    seen_add = seen.add
    deduped = sorted([i for i in alphanumeric_tokens if i not in seen and not seen_add(i)])
    fingerprint = ''.join(deduped)
    return fingerprint
df_dish['name_fingerprint']=df_dish['name2'].map(fingerprint)

In [14]:
# show clusters
clusters = df_dish.groupby('name_fingerprint')
result = clusters.size()
result.sort_values(ascending=False).head(10)

name_fingerprint
friedpotatoessweet       54
frenchfriedpotatoes      50
boiledpotatoes           48
2eggsonpoachedtoast      48
augratinpotatoes         46
dryextraghmumms          45
alecgingerimported       43
beefcoldroast            42
creamhashedinpotatoes    42
eggsfriedtwo             42
dtype: int64

In [20]:

# show the different friedpotatoessweet

df_dish.iloc[np.where(df_dish['name_fingerprint']=='friedpotatoessweet')]


,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price,name2,name_fingerprint
id,,,,,,,,,,
2089,Sweet Potatoes Fried,NaN,34,35,1892,1933,0.10,15.00,sweet potatoes fried,friedpotatoessweet
3780,"Sweet Potatoes, Fried",NaN,111,111,1881,1945,0.00,20.00,"sweet potatoes, fried",friedpotatoessweet
4075,fried Sweet Potatoes,NaN,418,425,1887,1984,0.00,30.00,fried sweet potatoes,friedpotatoessweet
9476,"Potatoes, sweet fried",NaN,3,3,1900,1933,0.30,0.30,"potatoes, sweet fried",friedpotatoessweet
17042,Fried sweet [potatoes],NaN,1,1,1900,1900,0.15,0.15,fried sweet [potatoes],friedpotatoessweet
17116,"Potatoes, Sweet, Fried",NaN,5,5,1900,1901,0.15,0.20,"potatoes, sweet, fried",friedpotatoessweet
38431,Sweet Potatoes - Fried,NaN,10,10,1900,1940,0.15,0.35,sweet potatoes - fried,friedpotatoessweet
42055,"Potatoes sweet, fried",NaN,1,1,1901,1901,0.15,0.15,"potatoes sweet, fried",friedpotatoessweet
56061,"Potatoes, fried sweet",NaN,12,12,1892,1913,0.15,0.25,"potatoes, fried sweet",friedpotatoessweet
